In [1]:
%config IPCompleter.greedy = True

In [2]:
import tensorflow as tf
from tensorflow import keras
from matplotlib import pyplot as plt
import numpy as np

physical_devices = tf.config.list_physical_devices('GPU') 
tf.config.experimental.set_memory_growth(physical_devices[0], True)

In [3]:
train_datagen = keras.preprocessing.image.ImageDataGenerator(rescale=1./255.)
validation_datagen = keras.preprocessing.image.ImageDataGenerator(rescale=1./255.)

In [4]:
train_generator = train_datagen.flow_from_directory(directory="../../TensorFlow数据集下载/tensorflow_datasets/horse-or-human/",
                                                    target_size=(200, 200),
                                                    batch_size=32,
                                                    class_mode="binary")

validation_generator = train_datagen.flow_from_directory(directory="../../TensorFlow数据集下载/tensorflow_datasets/validation-horse-or-human",
                                                    target_size=(200, 200),
                                                    batch_size=32,
                                                    class_mode="binary")

Found 1029 images belonging to 2 classes.
Found 256 images belonging to 2 classes.


In [5]:
model = keras.Sequential([
    keras.layers.Conv2D(16, (3, 3), activation="relu", input_shape=(300, 300, 3)),
    keras.layers.MaxPool2D(2, 2),
    keras.layers.Conv2D(32, (3, 3), activation="relu"),
    keras.layers.MaxPool2D(2, 2),
    keras.layers.Conv2D(64, (3, 3), activation="relu"),
    keras.layers.MaxPool2D(2, 2),
    keras.layers.Flatten(),
    keras.layers.Dense(512, activation="relu"),
    keras.layers.Dense(1, activation="sigmoid")
])

In [6]:
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 298, 298, 16)      448       
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 149, 149, 16)      0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 147, 147, 32)      4640      
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 73, 73, 32)        0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 71, 71, 64)        18496     
_________________________________________________________________
max_pooling2d_2 (MaxPooling2 (None, 35, 35, 64)        0         
_________________________________________________________________
flatten (Flatten)            (None, 78400)             0

In [7]:
model.compile(optimizer=keras.optimizers.RMSprop(lr=0.001),
              loss=keras.losses.binary_crossentropy,
              metrics=["accuracy"])

In [8]:
model.fit(train_generator, epochs=10, verbose=1, validation_data=validation_generator, validation_steps=8)

Epoch 1/10
33/33 [==============================] - 12s 265ms/step - loss: 3.4721 - accuracy: 0.6511 - val_loss: 0.7180 - val_accuracy: 0.8711
Epoch 2/10
33/33 [==============================] - 7s 217ms/step - loss: 0.1026 - accuracy: 0.9597 - val_loss: 2.2914 - val_accuracy: 0.7773
Epoch 3/10
33/33 [==============================] - 7s 208ms/step - loss: 0.0536 - accuracy: 0.9785 - val_loss: 6.0584 - val_accuracy: 0.7266
Epoch 4/10
33/33 [==============================] - 7s 209ms/step - loss: 0.9886 - accuracy: 0.9077 - val_loss: 2.2816 - val_accuracy: 0.8125
Epoch 5/10
33/33 [==============================] - 7s 206ms/step - loss: 0.0365 - accuracy: 0.9886 - val_loss: 2.3962 - val_accuracy: 0.8164
Epoch 6/10
33/33 [==============================] - 7s 205ms/step - loss: 0.0018 - accuracy: 1.0000 - val_loss: 3.6614 - val_accuracy: 0.7695
Epoch 7/10
33/33 [==============================] - 7s 205ms/step - loss: 0.2989 - accuracy: 0.9754 - val_loss: 7.0193 - val_accuracy: 0.7539
Epoch

In [12]:
from kerastuner.tuners import Hyperband
from kerastuner.engine.hyperparameters import HyperParameters

In [20]:
def build_model(hp):
    model = keras.Sequential()
    model.add(keras.layers.Conv2D(hp.Choice("num_filters_layer0", values=[16, 32, 64], default=16),
                                  (2, 2), activation="relu", input_shape=(200, 200, 3)))
    model.add(keras.layers.MaxPool2D(2, 2))
    for i in range(hp.Int("num_cov_layers", 1, 5)):
        model.add(keras.layers.Conv2D(hp.Choice(f"num_filters_layer{i+1}", values=[16, 32, 64], default=16), (2, 2), activation="relu"))
        model.add(keras.layers.MaxPool2D(2, 2))
    model.add(keras.layers.Flatten())
    model.add(keras.layers.Dense(hp.Int("hidden_units", 128, 512, step=32), activation="relu"))
    model.add(keras.layers.Dense(1, activation="sigmoid"))
    model.compile(optimizer=keras.optimizers.Adam(),
                  loss=keras.losses.binary_crossentropy,
                  metrics=["accuracy"])
    return model

In [21]:
hp = HyperParameters()
tuner = Hyperband(build_model,
                  objective="val_accuracy",
                  max_epochs=30,
                  directory="horse_human_params",
                  hyperparameters=hp,
                  project_name="my_horse_human_project")

In [22]:
tuner.search(train_generator, epochs=20, verbose=1, validation_data=validation_generator, validation_steps=8)

Trial 90 Complete [00h 02m 57s]
val_accuracy: 0.8203125

Best val_accuracy So Far: 0.90234375
Total elapsed time: 01h 00m 26s
INFO:tensorflow:Oracle triggered exit


In [23]:
best_hps = tuner.get_best_hyperparameters(30)[0]
print(best_hps.values)
model = tuner.hypermodel.build(best_hps)
model.summary()

{'num_filters_layer0': 64, 'num_cov_layers': 5, 'num_filters_layer1': 16, 'hidden_units': 288, 'num_filters_layer2': 16, 'num_filters_layer3': 64, 'num_filters_layer4': 64, 'tuner/epochs': 30, 'tuner/initial_epoch': 10, 'tuner/bracket': 3, 'tuner/round': 3, 'num_filters_layer5': 16, 'tuner/trial_id': '2c36313f4acd3e22dcf77077199e4527'}
Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 199, 199, 64)      832       
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 99, 99, 64)        0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 98, 98, 16)        4112      
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 49, 49, 16)        0         
________________________________________________

In [24]:
model.compile(optimizer=keras.optimizers.RMSprop(lr=0.001),
              loss=keras.losses.binary_crossentropy,
              metrics=["accuracy"])
model.fit(train_generator, epochs=10, verbose=1, validation_data=validation_generator)

Epoch 1/10
33/33 [==============================] - 14s 392ms/step - loss: 0.6451 - accuracy: 0.6408 - val_loss: 0.5419 - val_accuracy: 0.8438
Epoch 2/10
33/33 [==============================] - 6s 177ms/step - loss: 0.2825 - accuracy: 0.8977 - val_loss: 0.8845 - val_accuracy: 0.8477
Epoch 3/10
33/33 [==============================] - 6s 177ms/step - loss: 0.2023 - accuracy: 0.9268 - val_loss: 0.9789 - val_accuracy: 0.8281
Epoch 4/10
33/33 [==============================] - 6s 178ms/step - loss: 0.1547 - accuracy: 0.9294 - val_loss: 0.9244 - val_accuracy: 0.8086
Epoch 5/10
33/33 [==============================] - 6s 178ms/step - loss: 0.1251 - accuracy: 0.9534 - val_loss: 1.6446 - val_accuracy: 0.8164
Epoch 6/10
33/33 [==============================] - 6s 178ms/step - loss: 0.1217 - accuracy: 0.9529 - val_loss: 1.2380 - val_accuracy: 0.8594
Epoch 7/10
33/33 [==============================] - 7s 208ms/step - loss: 0.0756 - accuracy: 0.9753 - val_loss: 0.7850 - val_accuracy: 0.8789
Epoch

In [11]:
best_hps